# Pipe 3

This notebook contains scripts for calculating kinematic data.

Input files must be `.csv` files generated by `rtklib-pos-to-csv.ipynb`.
Its headers are defined in the code below.

Outputs another `.csv` file _with the **different** headers_.
Those new headers are also defined in the code below.

In [33]:
import csv
import datetime as dt
import math
import pandas as pd
from geopy.distance import geodesic
from constants import *
from world import *
from scipy.interpolate import Akima1DInterpolator
from scipy.interpolate import CubicSpline
from scipy.interpolate import PchipInterpolator

In [34]:
INPUT_FILEPATH = "/home/delaynie/Documents/rtklib-workspace/cmubuggy-datalogger/data/position-csv/"
INPUT_FILENAMES = ["output1.csv"] #, "output2.csv", "output3.csv"]
OUTPUT_FILEPATH = "/home/delaynie/Documents/rtklib-workspace/cmubuggy-datalogger/data/calc-csv/"

In [35]:
MIN_SATELLITES = 2
LPF_COEF = 0.5 # coefficient on previous values

In [36]:
TIMESTAMP_FORMAT = "%Y/%m/%d %H:%M:%S.%f"

# TODO make this compatible with several levels of precesion
# converts timestamp string to UTC value (float)
def timestampToValue(ts: str):
    return dt.datetime.strptime(ts, TIMESTAMP_FORMAT).timestamp()

In [37]:
def getMagnitude(vector):
    return math.sqrt(math.pow(vector[0], 2) + math.pow(vector[1], 2))

In [38]:
"""
Code yoinked from https://github.com/CMU-Robotics-Club/RoboBuggy2/blob/sw/20230331/rb_ws/src/buggy/scripts/auton/world.py

    Abstraction for the world coordinate system

    The real world uses GPS coordinates, aka latitude and longitude. However,
    using lat/lon is bad for path planning for several reasons. First, the difference
    in numbers would be very tiny for small distances, alluding to roundoff errors.
    Additionally, lat/lon follows a North-East-Down coordinate system, with headings
    in the clockwise direction. We want to use an East-North-Up coordinate system, so
    that the heading is in the counter-clockwise direction.

    We do this by converting GPS coordinates to UTM coordinates, which are in meters.
    UTM works by dividing the world into a grid, where each grid cell has a unique
    identifier. A UTM coordinate consists of the grid cell identifier and the "easting"
    and "northing" within that grid cell. The easting (x) and northing (y) are in meters,
    and are relative to the southwest corner of the grid cell.

    Last, we offset the UTM coordinates to be relative to some arbitrary zero point. That
    way, the final world coordinates are relatively close to zero, which makes debugging
    easier.

    This class provides methods to convert between GPS and world coordinates. There is
    a version for single coordinates and a version for numpy arrays.
"""

def gps_to_world(lat, lon):
    """Converts GPS coordinates to world coordinates

    Args:
        lat (float): latitude
        lon (float): longitude

    Returns:
        tuple: (x, y) in meters from some arbitrary zero point
    """
    utm_coords = utm.from_latlon(lat, lon)
    x = utm_coords[0] - World.WORLD_EAST_ZERO
    y = utm_coords[1] - World.WORLD_NORTH_ZERO

    return x, y

In [39]:
def calc(inputPath, outputPath):
    input_df = pd.read_csv(inputPath)

    ### Converting to linear local coordinatess
    local_coords = []
    for i in range(len(input_df)):
        local_coords.append(gps_to_world(input_df["latitude"][i], input_df["longitude"][i]))

    # Adding the linear local coords to input_df
    input_df["latitude_meters"] = [coord[0] for coord in local_coords]
    input_df["longitude_meters"] = [coord[1] for coord in local_coords]

    ### Applying a LPF to the linear local coords for curve smoothing
    # for i in range(len(input_df)):
    #     if i != 0:
    #         input_df["latitude_meters"][i] = (LPF_COEF)*input_df["latitude_meters"][i-1] + (1-LPF_COEF)*input_df["latitude_meters"][i]
    #         input_df["longitude_meters"][i] = (LPF_COEF)*input_df["longitude_meters"][i-1] + (1-LPF_COEF)*input_df["longitude_meters"][i]

    ### Setting up the interpolator
    interpolator = PchipInterpolator(input_df.loc[:, "timestamp"].map(timestampToValue), input_df.loc[:, ["latitude_meters", "longitude_meters"]])
    positions = interpolator(input_df.loc[:, "timestamp"].map(timestampToValue), nu=0)
    velocities = interpolator(input_df.loc[:, "timestamp"].map(timestampToValue), nu=1)
    accelerations = interpolator(input_df.loc[:, "timestamp"].map(timestampToValue), nu=2)

    output_df = pd.DataFrame(columns=CALC_CSV_HEADER)

    output_df["timestamp"] = input_df.loc[:, "timestamp"].map(timestampToValue)
    output_df["timestamp_str"] = input_df["timestamp"]

    output_df["fix_type"] = input_df["fix-type"]

    output_df["latitude"] = input_df["latitude"]
    output_df["longitude"] = input_df["longitude"]
    output_df["altitude"] = input_df["altitude"]

    output_df["latitude_processed"] = [coord[0] for coord in positions]
    output_df["longitude_processed"] = [coord[1] for coord in positions]

    output_df["velocity"] = velocities.tolist()
    output_df["speed"] = [getMagnitude(vector) for vector in velocities]

    output_df["acceleration_vector"] = accelerations.tolist()
    output_df["acceleration_magnitude"] = [getMagnitude(vector) for vector in accelerations]

    output_df.to_csv(outputPath)

In [40]:
for name in INPUT_FILENAMES:
    calc(INPUT_FILEPATH+name, OUTPUT_FILEPATH+name)